In [94]:
import sys
import os
from  os import  path
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from dataclasses import dataclass
import json
from tqdm import tqdm
import plotly.graph_objs as go
import pickle
pd.set_option('display.max_columns', 500)

from PIL import Image
mpl.font_manager._rebuild()
plt.rcParams['figure.figsize'] = (16 ,9)
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
import seaborn as sns

In [81]:
import re
def floor_int(floor_str):
    try:
        floor_int = int(re.search("\d+", floor_str).group())
    except AttributeError:
        return -9999
    if (floor_str[-1] == "B") or (floor_str[0] == "B"):
        return int(floor_int * (-1))
    else:
        return int(floor_int -1)
floor_int("B12")

-12

In [98]:
def build_master():
    df_dirs = os.listdir("../dataframes/")
    for df_dir in df_dirs:
        paths = path.join("../dataframes/", df_dir)
        df = pd.DataFrame()
        site_data = {}
        for (i,p) in enumerate(os.listdir(paths)):
            floor = p.split("_")[1]
            floor = floor.split(".")[0]
            with open(os.path.join(paths,  p), 'rb') as f:
                data_dict = pickle.load(f)
                for key in ["waypoint_df", "acce_df", "gyro_df", "magn_df", "rot_df", "wifi_df", "ibeacon_df"]:
                    data_dict[key]["floor"] = floor_int(floor)
                    if key == "wifi_df":
                        data_dict[key].rename(columns={'rssi': 'wifi_rssi'}, inplace=True)
                    elif key == "ibeacon_df":
                        data_dict[key].rename(columns={'rssi': 'ibeacon_rssi'}, inplace=True)
                    if key in site_data:
                        site_data[key] = pd.concat([site_data[key], data_dict[key]], axis=0)
                    else:
                        site_data[key] = data_dict[key]
        site_master_df = site_data["acce_df"]
        site_master_df = pd.merge(site_master_df, site_data["waypoint_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        site_master_df = pd.merge(site_master_df, site_data["gyro_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        site_master_df = pd.merge(site_master_df, site_data["magn_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        site_master_df = pd.merge(site_master_df, site_data["rot_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        site_master_df = pd.merge(site_master_df, site_data["wifi_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        site_master_df = pd.merge(site_master_df, site_data["ibeacon_df"], on =[ "timestamp", "path_name", "floor"], how='outer')
        
        float_cols = ['acce_x', 'acce_y', 'acce_z','waypoint_x', 'waypoint_y', 'gyro_x', 'gyro_y', 'gyro_z', 'magn_x', 'magn_y', 'magn_z', 'rot_x', 'rot_y', 'rot_z']
        site_master_df.loc[:, float_cols] = site_master_df.loc[:, float_cols].astype(np.float32)
        categorical_cols = ["path_name",  "ssid", "bssid","uuid_major_minor"]
        site_master_df.loc[:, categorical_cols] = site_master_df.loc[:, categorical_cols].astype("category")
        
        wifi_df = site_master_df.loc[:, ["timestamp", "ssid","bssid", "wifi_rssi"]].dropna()
        waypoint_df = site_master_df.loc[:, ["timestamp", "waypoint_x","waypoint_y", "floor"]].dropna()
        # site_master_df["floor"] = list(map(floor_int, site_master_df["floor"]))
        
        
        display(site_master_df)
        return site_master_df, waypoint_df, wifi_df
    
master, waypoint, wifi = build_master()

,timestamp,acce_x,acce_y,acce_z,path_name,floor,waypoint_x,waypoint_y,gyro_x,gyro_y,gyro_z,magn_x,magn_y,magn_z,rot_x,rot_y,rot_z,ssid,bssid,wifi_rssi,lastseen_ts,uuid_major_minor,ibeacon_rssi
0,2019-10-22 18:06:18.949,-0.210388,-0.037125,5.890442,5daec8a5aa1d300006faafdd.txt,7,NaN,NaN,0.960800,-0.086578,0.092148,-0.279236,-32.221985,-27.900696,0.019756,0.018009,-0.999419,NaN,NaN,NaN,NaT,NaN,NaN
1,2019-10-22 18:06:18.969,0.289413,-0.092804,4.514984,5daec8a5aa1d300006faafdd.txt,7,NaN,NaN,0.535217,-0.279388,0.092148,-3.053284,-32.221985,-27.900696,0.017190,0.011522,-0.999541,NaN,NaN,NaN,NaT,NaN,NaN
2,2019-10-22 18:06:18.989,0.601257,-0.143677,3.447754,5daec8a5aa1d300006faafdd.txt,7,NaN,NaN,0.085678,-0.353958,0.045273,-1.666260,-32.916260,-27.230835,0.009830,0.010478,-0.999691,NaN,NaN,NaN,NaT,NaN,NaN
3,2019-10-22 18:06:19.009,0.786804,-0.214905,2.492477,5daec8a5aa1d300006faafdd.txt,7,NaN,NaN,-0.272797,-0.182449,-0.025040,-0.971985,-34.303284,-25.889587,0.002205,0.013504,-0.999781,NaN,NaN,NaN,NaT,NaN,NaN
4,2019-10-22 18:06:19.029,1.060944,-0.456131,2.089645,5daec8a5aa1d300006faafdd.txt,7,NaN,NaN,-0.675995,-0.017334,-0.086823,-0.971985,-34.303284,-27.230835,-0.004730,0.021094,-0.999719,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55544,2019-10-22 11:25:59.908,NaN,NaN,NaN,5dae695daa1d300006faac13.txt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,89cb11b04122cef23388b0da06bd426c1f48a9b5_cf6a3...,-74.0
55545,2019-10-22 11:26:00.066,NaN,NaN,NaN,5dae695daa1d300006faac13.txt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,89cb11b04122cef23388b0da06bd426c1f48a9b5_cf6a3...,-76.0
55546,2019-10-22 11:26:00.184,NaN,NaN,NaN,5dae695daa1d300006faac13.txt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,9158ffb381be1c3132a80ea9a8674cd813a8eee4_879a7...,-89.0
55547,2019-10-22 11:26:00.219,NaN,NaN,NaN,5dae695daa1d300006faac13.txt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,89cb11b04122cef23388b0da06bd426c1f48a9b5_cf6a3...,-78.0


In [100]:
waypoint

,timestamp,waypoint_x,waypoint_y,floor
283,2019-10-22 18:06:24.569,58.092224,13.257805,7
542,2019-10-22 18:06:29.712,51.953407,12.937272,7
1852,2019-10-22 18:06:49.787,44.529961,25.705687,7
13534,2019-10-22 16:53:21.437,45.614220,69.602074,5
18496,2019-10-22 18:06:18.765,60.900169,14.035721,7
18497,2019-10-22 18:06:20.387,60.978004,12.135031,7
18498,2019-10-22 18:06:26.664,55.889259,13.336100,7
18499,2019-10-22 18:06:33.267,48.442684,13.507691,7
18500,2019-10-22 18:06:36.129,45.144772,12.922127,7
18501,2019-10-22 18:06:42.578,44.952988,20.227160,7
